# Kale <3 Parsl

<img src=https://www.fodmapeveryday.com/wp-content/uploads/2017/12/fennel-kale-and-parsley-for-salad.jpg width=500px/>
<br />

Here, we demonstrate using Kale to visualize and delay execution of Parsl workflows.

### Two-way communication
- Kale intercepts Parsl execution by overriding the Parsl DataFlowKernel
- Kale executes tasks via Parsl

### Scope
The following is a brief demonstration of this interaction. So far, only Python functions have been implemented. I haven't yet tried to integrate bash functions or batch tasks as we have in Kale.

### Passing Data in Kale
To match the Parsl model, it was necessary to allow data to be passed from one Kale Task to another via arguments, which was not previously possible. So that's a cool new feature!

For example, if the output of `Kale Task A` should be an argument to `Kale Task B`, then `Kale Task A` itself should be passed as an argument, and it's result will be evaluated immediately before `Kale Task B` is executed.

### Breaking changes
Also, I've made some changes to achieve this which have probably broken the previous Fireworks model, so some modifications will be necessary before merging to master.

In [1]:
from parsl import DataFlowKernel, ThreadPoolExecutor, App

from kale.workflow_widgets import WorkflowWidget, WorkerPoolWidget
from kale.parsl_dflow import KaleDFK

# Initialization

The `WorkerPoolWidget` defines how Kale tasks will be executed. The interface is definitely in progress. For example, the `Location` field doesn't do anything yet, but could be used to specify that a Workflow should execute on remote resources.

`Workers` is passed to `max_workers` in the internal ThreadPoolExecutor which will Parsl will ultimately use to execute the tasks.

In [2]:
WorkerPoolWidget?

Init signature: WorkerPoolWidget(*args, **kwargs)
Docstring:      GUI widget for managing WorkerPools.
Init docstring: Public constructor
File:           /opt/kale/kale/workflow_widgets.py
Type:           MetaHasTraits


In [3]:
wpw = WorkerPoolWidget()
wpw

INIT WPW


WorkerPoolWidget(children=(HTML(value='<h3>Worker Pools</h3>'), TableWidget(children=(RowWidget(children=(HBox(children=(HTML(value='<b>Name</b>'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='<b>Location</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Executor</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Workers</b>'),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(HTML(value='<b>Action</b>'),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(HTML(value='default'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='4'),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='danger', description='Remove', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(Text(value='', layout=Layout(width='148px')),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('localhost',), value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('parsl', 'fireworks'), value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(IntText(value=1, layout=Layout(width='198px')),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='success', icon='plus', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px'))), layout=Layout(width='710px')), HTML(value='\n        <div class="alert alert-success" style="width: 710px">\n        WorkerPool \'default\' created.\n        </div>\n        ')))

In [13]:
wpw.out_area

Output()

Initialize the Kale `DataFlowKernel`. I don't think this `ThreadPoolExecutor` is actually used. This interface can be cleaned up.

In [4]:
workers = ThreadPoolExecutor()
kale_dfk = KaleDFK(executors=[workers])

# Define Parsl function

Define a Parsl function as normal, except the `KaleDFK` is passed in place of the standard Parsl DFK.

In [5]:
@App('python', kale_dfk)
def rand_add(*prev_list):
    """Add a random number to the previous ones."""
    import random
    import time
    
    # Random int between 0 & 10, inclusive.
    myrand = random.randint(0, 10)
    mysum = myrand + sum(prev_list)
    
    print("My number is {}. I was given {}. The sum is {}.\n".format(myrand, prev_list, mysum), end='')
    time.sleep(2)
    
    return mysum


# Define workflow
Secify the name of the new workflow. By requiring that a Workflow be initialized, multiple workflows can be defined sequentially with the same `KaleDFK` object by calling this function in between workflow definitions.

In [6]:
kale_dfk.new_workflow('Random Tree')

Now, instead of executing the tasks, the `KaleDFK` intercepts them and combines them into a kale `Workflow`. As you can see, the only result of this intercepted execution is a Parsl `AppFuture` returning `None`

In [7]:
rand_add(
    rand_add(
        rand_add(),
        rand_add()
    ),
    rand_add(
        rand_add(),
        rand_add()
    )
)

rand_add(
    rand_add(
        rand_add(),
        rand_add()
    ),
    rand_add(
        rand_add(),
        rand_add()
    )
)

<AppFuture at 0x7f62523dc588 state=finished returned NoneType>

# Launch workflow
A workflow widget is used to visualize and interact with the workflow once it has been defined.

Tasks can be selected in the plot using the mouse. `ctrl-click` for multi-select.

### `Workflow`
The **Run Workflow** button in the `Workflow` tab starts execution. 
If some tasks are selected, only those tasks are run. Currently, an error will be raised if there's an unselected task in the middle of a dependency chain.

Also, this is likely to cause some issues in the case of data being passed explicitly between functions as we have here.

### `Task`
In the `Task` tab, some basic information about the selected task is displayed, including the function name and its arguments.

### `Tags`
In the `Tags` tab, various selection utilites are available. Also, we have the concept of assigning tags to Kale tasks for easier selection and grouping. I haven't implemented this for workflows generated from Parsl.

### `Widget Log`
The `Widget Log` tab is generally where output from the functions is displayed, but I currently have it disabled, so all output will be printed below the widget.

In [8]:
ww = WorkflowWidget(kale_dfk.kale_workflow, wpw)
ww

WorkflowWidget(children=(VBox(children=(Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 2}, {'source': 1, 'target': 2}, {'source': 2, 'target': 6}, {'source': 3, 'target': 5}, {'source': 4, 'target': 5}, {'source': 5, 'target': 6}, {'source': 7, 'target': 9}, {'source': 8, 'target': 9}, {'source': 9, 'target': 13}, {'source': 10, 'target': 12}, {'source': 11, 'target': 12}, {'source': 12, 'target': 13}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '1', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '2', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7f62523cfe80>, <kale.workflow_objects.PythonFunctionTask object at 0x7f62523cfb38>]', 'kwargs': '{}'}, {'label': '3', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '4', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '5', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7f62523cf240>, <kale.workflow_objects.PythonFunctionTask object at 0x7f62523e16a0>]', 'kwargs': '{}'}, {'label': '6', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7f62523cf710>, <kale.workflow_objects.PythonFunctionTask object at 0x7f62523e1668>]', 'kwargs': '{}'}, {'label': '7', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '8', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '9', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7f62523dbcf8>, <kale.workflow_objects.PythonFunctionTask object at 0x7f62523db780>]', 'kwargs': '{}'}, {'label': '10', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '11', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '12', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7f6285701e48>, <kale.workflow_objects.PythonFunctionTask object at 0x7f62523dfdd8>]', 'kwargs': '{}'}, {'label': '13', 'shape': 'rect', 'name': 'rand_add', 'task_

# View results
Results of individual Kale Tasks are stored as futures on the task object. One way of accessing the result by task number (as shown in the DAG visualization) is shown below. There's probably a nicer way to do this.

In [9]:
kale_dfk.kale_workflow.get_future(6).result()

KeyboardInterrupt: 

We also have a `kale_workflow.get_task_by_name(name)`, but in this case, all tasks have the same name since they all come from the same Parsl App, so that won't work here.

## Kale-Parsl Improvements
- Only Python tasks are currently supported with Parsl. This should be expanded to bash and batch tasks.
- In the same vein, I'm not sure whether IPyParallel execution of Python tasks will work. It should be doable with a few small changes.
- In the spirit of Parsl's on-the-fly workflow definition, it should be possible to dynamically update the widget if new tasks are added to the workflow. This isn't yet implemented.
- Error handling is a bit obfuscated. This could definitely be improved somehow.

## Kale Improvements
- We're in the process of developing a Kale service which will facilitate monitoring and controlling the status of workflows and tasks between multiple notebooks and across resources.
- General UI improvements are underway. Suggestions are welcome!